# PM2.5 - Local Conditions in Arapahoe County, Colorado

PM2.5 refers to atmospheric particulate matter (PM) that have a diameter of less than 2.5 micrometers, which is about 3% the diameter of a human hair.

Fine particles can come from various sources. They include power plants, motor vehicles, airplanes, residential wood burning, forest fires, agricultural burning, volcanic eruptions and dust storms.

This increases the chances of humans and animals inhaling them into the bodies. Owing to their minute size, particles smaller than 2.5 micrometers are able to bypass the nose and throat and penetrate deep into the lungs and some may even enter the circulatory system. Studies have found a close link between exposure to fine particles and premature death from heart and lung disease. Fine particles are also known to trigger or worsen chronic disease such as asthma, heart attack, bronchitis and other respiratory problems.


On a very clear and non-hazy day, the PM2.5 concentration can be as low as 5 μg/m3 or below. The 24-hour concentration of PM2.5 is considered unhealthy when it rises above 35.4 μg/m3.

*Source:https://blissair.com/what-is-pm-2-5.htm*

In [ ]:
import requests
import json
import pandas as pd
from matplotlib import pyplot
from matplotlib.lines import Line2D
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tools.eval_measures import rmse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from math import sqrt
import datetime
import itertools
from pmdarima.arima import auto_arima
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (20,10)

import random

random.seed( 156464 )

## Loading data from Arapahoe County
#### Using EPA's API by County for PM2.5 levels

EPA.Gov has a wide range of resources that help track various air quality stats. EPA's API key is easy to obtain. Cannot load more than a year at a time. 

*Documentation for EPA's API: https://aqs.epa.gov/aqsweb/documents/data_api.html*


In [ ]:
#API inputs
email = " " #insert email here
key = " " #insert key here
api_url = "https://aqs.epa.gov/data/api/sampleData/byCounty"

json_list = []
#data collection did not start for this area until 1999
for year in range(1999,2020):
    param = {'email': email, 
             'key': key,
             'param':'88101', #code for the PM2.5 levels
             'bdate':'%s0101' %year, 
             'edate':'%s1231' %year,
             'state':'08', #State code for Colorado
             'county':'005'} #County code for Arapahoe County
    data = requests.get(api_url, params = param)
    resolvedwo = data.json()
    json_list.append(resolvedwo)

with open('resolvedworesolution.json', 'w+') as f:
    json.dump(json_list, f, sort_keys=True, indent=4)


In [ ]:
json_list

## Extact Values from the Json file
The main data we want to track is the PM2.5 levels through time. We will need Date and Measurement of the PM2.5 levels    Regardless of where the key lives in the JSON, this extract_values function returns every value for the instance of "key." 

*Source of function: https://hackersandslackers.com/extract-data-from-complex-json-python/*

In [ ]:
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

local_date = extract_values(json_list, 'date_local')
sample_measurement = extract_values(json_list, 'sample_measurement')

## Data Preprocessing and Visualization

In [ ]:
#creating a dataframe that has both the local date and sample measurement 
data = pd.DataFrame(list(zip(local_date, sample_measurement)),
              columns=['local_date','sample_measurement'])
data.local_date = pd.to_datetime(data.local_date, format="%Y-%m-%d")
#sort the dates in order
data.sort_values(by=['local_date'], inplace=True, ascending=True)
#the range of the PM2.5 through the last 10 years
data.describe() 

In [ ]:
data.info() #see how many non-null rows are in each column

### Data cleaning
Looks like there are some large outliers and missing values in the dataframe. Max value in the PM2.5 samples is 140.4, which seems like an error. 
Also there are 123 NAN values. I am going to replace it with the median so that models do not have any errors in the time series analysis. Not going to use mean, as the data is right skewed. 

In [ ]:
median = data['sample_measurement'].median()
#replace any outliers where it's greater than 100 to the median
data.loc[data.sample_measurement > 100, 'sample_measurement'] = median
#replace all NAN values with the median
data['sample_measurement']= data['sample_measurement'].fillna(value = median)
data.describe()

In [ ]:
#As this is time series data, the date column will be set as the index
data = data.set_index('local_date')
data.head()

### Data Visualization

In [ ]:
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (20,10)
pyplot.scatter(data.index, data.sample_measurement, marker = 'o')
#adding a line at the 35PM2.5 levels, which the EPA deemed unsafe
pyplot.axhline(y=35, color='r', linestyle='-')
pyplot.legend([Line2D([0],[0], color = 'red', linewidth =3, linestyle = '-')], ['Dangerous Levels'], fontsize = 15)

pyplot.rc('xtick', labelsize=15)    # fontsize of the tick labels
pyplot.rc('ytick', labelsize=15)    # fontsize of the tick labels
pyplot.title('PM2.5 Levels in Arapahoe County', fontsize=30)
pyplot.show()

Looks like there are some days where the PM2.5 levels in Arapahoe County have exceeded safe levels. However, there seems to be no trend in PM2.5 levels. 

In [ ]:
data['year'] = pd.DatetimeIndex(data.index).year
data.head()
data.boxplot(by ='year', column =['sample_measurement'], grid = False) 
pyplot.title('PM2.5 Levels in Arapahoe County by Year Boxplot', fontsize=30)
pyplot.show()

The boxplot of every year demonstrates that the median trend of the data has remained relatively flat. Also the number of outliers are also consistent. 

## LSTM for Univariate Time Series Prediction

I want to create a model that can predict the next 6 months of PM2.5 levels in the county. The sample is taken every 3 days, which would mean I would need to predict ~60 steps ahead. This is how I'll be training my model to see how well it does if I hold back 6 months of data.


In [ ]:
data = data.drop('year', 1)
steps = 60 #60 steps represents ~6 months of data. 60*3days = 180days
train, test = data[:-steps], data[-steps:] 

'''
Because the data is not trending up or down, and I have included outlier mix and max values (from boxplot), 
the MinMaxScaler should be an appropriate scaler. 
'''
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

### Tuning the LSTM model
Going to create a tuning grid for the lstm model. The tuning parameters I want to adjust are the length param, number of starter nodes in the LSTM layer, dropout rate in the Dropout layer, and the number of epochs used to train the model. 
You can also change the activation and optimizer parameters to see if the tuning would work as well. 

*Note:* My compute could only handle ~150 iterations of the tuning grid. Therefore, in the first iteration, I have a diluted the parameters. If I was going to care about getting it closer, I would do this a couple of time, getting more narrow on the ranges for the best preformers. 

In [ ]:
lstm_grid = {'n_input' : np.arange(2,63,60), #how many timesteps to use for prediction (for the length param in generator)
             'batch_size' : np.arange(1,22,20),  #how samples will be taken from the train data (for the generator)
             'nodes' : np.arange(50,1051,500), #how many intial nodes use for the LSTM layer
            'dropout': np.arange(0,.5,.2), #what dropout rate to use for the Dropout layer for overfitting
             'epochs' : np.arange(10,1011,1000)} #how many epochs to use in the fit 

In [ ]:
order = lstm_grid.keys()
tuning_grid = pd.DataFrame(itertools.product(*[lstm_grid[k] for k in order]), columns=order)
tuning_grid

In [ ]:
m = len(tuning_grid)
m

Need to create an empty list for the RMSE errors that will come out of the prediction. I'm using RMSE here because this will compare the actual sample measurements to the predictions. Therefore, the RMSE will be easier to interpret. The list will be appended to the tuning grid dataframe to see which tuning parameter combinations resulted in the lowest RMSE. 

In [ ]:
lstm_rmse = [None]*m

In [ ]:
'''
This for loop will go through each combination in the tuning grid. It will make a LSTM model, a prediction 
and calculate the RMSE between the actual data and the prediction. It will save off the RMSE where I can choose
the best result (lowest). 
'''

for i in range(m):
    n_features = 1
    tmp_n_input = tuning_grid.n_input[i]
   #Time Series Generator
    tmp_generator = TimeseriesGenerator(data = train, targets = train, 
                                        length = tmp_n_input, 
                                        batch_size = tuning_grid.batch_size[i])
   #LSTM Model
    tmp_model = Sequential()
    tmp_model.add(LSTM(tuning_grid.nodes[i], 
                   activation = 'relu', 
                   input_shape =(tmp_n_input, n_features)))
    tmp_model.add(Dropout(tuning_grid.dropout[i]))
    tmp_model.add(Dense(1))
    tmp_model.compile(optimizer = 'adam', loss = 'mse')

    tmp_model.fit_generator(tmp_generator, steps_per_epoch =1, epochs=tuning_grid.dropout[i])

    #Prediction
    tmp_x_input = train[-tmp_n_input:].reshape((1, tmp_n_input, n_features))

    tmp_pred_list = []

    for j in range(steps):
        #one step ahead prediciton
        tmp_pred_list.append(tmp_model.predict(tmp_x_input)[0]) 
        #append the one-step prediction to the x_input used to forecast the following step
        tmp_x_input = np.append(tmp_x_input[:, 1:, :], [[tmp_pred_list[j]]], axis = 1) 

    tmp_df_predict = pd.DataFrame(scaler.inverse_transform(tmp_pred_list), 
                                  index = data[-steps:].index, columns = ["tmp_predictions"])
    tmp_df_test = pd.concat([data, tmp_df_predict], axis = 1)

    lstm_rmse[i] = sqrt(mean_squared_error(tmp_df_test.sample_measurement[-steps:], 
                                           tmp_df_test.tmp_predictions[-steps:]))

    #To track the progress 
    print(i,"\n")

In [ ]:
tuning_results = pd.concat([tuning_grid, pd.DataFrame(lstm_rmse, columns = ['lstm_rmse'])], axis = 1)
tuning_results

In [ ]:
#see which combination had the lowest rmse
tuning_results[['lstm_rmse']].idxmin() 

In [ ]:
#best results were at index 70
tuning_results.loc[70]

In [ ]:
best_result_index = 70

### Final LSTM model based off the best preforming tuning parameters

In [ ]:
n_features = 1
n_input = tuning_grid.n_input[best_result_index]
#Time Series Generator
generator = TimeseriesGenerator(data = train, targets = train, 
                                length = n_input, 
                                batch_size = tuning_grid.batch_size[best_result_index])
#LSTM Model
model = Sequential()
model.add(LSTM(tuning_grid.nodes[best_result_index], 
                   activation = 'relu', 
                   input_shape =(n_input, n_features)))
model.add(Dropout(tuning_grid.dropout[best_result_index]))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mse')

model.fit_generator(generator, steps_per_epoch =1, epochs=tuning_grid.dropout[best_result_index])

#Prediction
x_input = train[-n_input:].reshape((1, n_input, n_features))

pred_list = []

for j in range(steps):
    #one step ahead prediciton
    pred_list.append(model.predict(x_input)[0]) 
    #append the one-step prediction to the x_input used to forecast the following step
    x_input = np.append(x_input[:, 1:, :], [[pred_list[j]]], axis = 1) 

df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index = data[-steps:].index, columns = ["predictions"])

df_test = pd.concat([data, df_predict], axis = 1)

lstm_rmse = sqrt(mean_squared_error(df_test.sample_measurement[-steps:], df_test.predictions[-steps:]))
print('The rmse is', lstm_rmse)

In [ ]:
df_test.tail()

#### Visualize the results 

In [ ]:
pyplot.plot(df_test.index[-(steps+60):], df_test.sample_measurement[-(steps+60):])
pyplot.plot(df_test.index[-(steps+60):], df_test.predictions[-(steps+60):], color = 'r')
pyplot.show()

Calculating the RMSE between the prediction and actual 

### Future Predictions

In [ ]:
#make training data the whole dataframe
future_predict_train = data

scaler.fit(future_predict_train)
future_predict_train = scaler.transform(future_predict_train)

future_predict_generator = TimeseriesGenerator(data = future_predict_train, targets = future_predict_train, 
                                length = n_input, batch_size = tuning_grid.batch_size[best_result_index])

#using the same model as above
model.fit_generator(future_predict_generator, steps_per_epoch = 1, epochs= tuning_grid.dropout[best_result_index])

future_x_input = future_predict_train[-n_input:].reshape((1, n_input, n_features))

future_predict_pred_list = []

for i in range(steps):
    #one step ahead prediciton
    future_predict_pred_list.append(model.predict(future_x_input)[0]) 
    #append the one-step prediction to the x_input used to forecast the following step
    future_x_input = np.append(future_x_input[:, 1:, :], [[future_predict_pred_list[i]]], axis = 1) 

Adding future dates for the prediction. Because the sample is taken every 3 days, the range has to be increased by 3

In [ ]:
add_dates = [data.index[-1] + datetime.timedelta(days=x) for x in range(0,steps*3+1,3)]
future_dates = pd.DataFrame(index = add_dates[1:], columns = data.columns)
future_dates.head()

In [ ]:
df_future_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                                index= future_dates[-steps:].index, columns = ['prediction'])

In [ ]:
df_proj = pd.concat([data, df_future_predict], axis = 1)

In [ ]:
df_proj.tail()

#### Visualize the Results 

In [ ]:
pyplot.plot(df_proj.index[-(steps+60):], df_proj.sample_measurement[-(steps+60):])
pyplot.plot(df_proj.index[-(steps+60):], df_proj.prediction[-(steps+60):], color = 'r')
pyplot.show()

## ARIMA Model for comparison 
Usuallly ARIMA can model univariate time series models quiet well. Interested in seeing how well it preforms against LSTM. 

In [ ]:
steps = 60 #going to keep the same amount of steps for comparison 
train, test = data[:-steps], data[-steps:] 

In [ ]:
#building the model

model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)

forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast ,index = test.index,columns=['Prediction'])

In [ ]:
df_proj = pd.concat([data, forecast], axis = 1)
df_proj.tail()

In [ ]:
rmse = sqrt(mean_squared_error(test,forecast))
print("ARMIA's RMSE is:", rmse)

#### Visualize the Results 

In [ ]:
pyplot.plot(train[-(steps+60):], label='Train')
pyplot.plot(test, label='Test')
pyplot.plot(forecast, label='Prediction')
pyplot.legend(fontsize = 15)
pyplot.show()